In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf

2023-10-22 05:52:40.806335: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-22 05:52:40.809280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-22 05:52:40.868835: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-22 05:52:40.870273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 05:52:41.939869: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# Importing data

In [4]:
df = pd.read_excel("./output1k.xlsx")
df.head()

,formula_pretty,formula_anonymous,chemsys,material_id,structure,formation_energy_per_atom,is_metal,e_total,e_ionic,e_electronic,...,e_total_free,ef,e_band/num_atoms,e_coulomb/num_atoms,e_entropy/num_atoms,e_external/num_atoms,e_kinetic/num_atoms,e_kinetic0/num_atoms,e_xc/num_atoms,e_total_free/num_atoms
0,YCrB4,ABC4,B-Cr-Y,mp-20450,Full Formula (Y4 Cr4 B16)\nReduced Formula: YC...,-0.653251,False,47.348925,5.890129,41.458796,...,-5.858164,11.163444,0.024319,-0.188020,-2.767267e-05,0,0.221036,0.196718,-0.279148,-0.244090
1,LiCaGaF6,ABCD6,Ca-F-Ga-Li,mp-12829,Full Formula (Li2 Ca2 Ga2 F12)\nReduced Formul...,-3.463849,False,6.967909,4.827025,2.140884,...,-2.640814,3.665931,-1.747124,-0.837262,-1.270295e-12,0,0.958238,2.705362,-0.282488,-0.146712
2,Na2GeO3,AB2C3,Ge-Na-O,mp-5784,Full Formula (Na4 Ge2 O6)\nReduced Formula: Na...,-2.140016,False,11.718332,8.805022,2.913310,...,-1.935350,3.721439,-1.917246,-0.664702,-5.033265e-08,0,0.767800,2.685047,-0.275572,-0.161279
3,CsMgCl3,ABC3,Cl-Cs-Mg,mp-23004,Full Formula (Cs2 Mg2 Cl6)\nReduced Formula: C...,-2.322267,False,10.490509,7.869904,2.620605,...,-0.992013,4.640916,-3.683055,-0.640347,-2.584534e-12,0,0.733330,4.416384,-0.197104,-0.099201
4,Li2VSiO5,ABC2D5,Li-O-Si-V,mp-18860,Full Formula (Li8 V4 Si4 O20)\nReduced Formula...,-2.766672,False,16.969145,13.458970,3.510175,...,-3.878814,6.460653,-1.758132,-0.449097,-5.588154e-04,0,0.499584,2.257716,-0.272984,-0.215490


In [5]:
df.shape

(996, 30)

In [6]:
df.dtypes

formula_pretty                object
formula_anonymous             object
chemsys                       object
material_id                   object
structure                     object
formation_energy_per_atom    float64
is_metal                        bool
e_total                      float64
e_ionic                      float64
e_electronic                 float64
n                            float64
fields_not_requested          object
structure_cif                 object
e_band                       float64
e_coulomb                    float64
e_entropy                    float64
e_external                     int64
e_kinetic                    float64
e_kinetic0                   float64
e_xc                         float64
e_total_free                 float64
ef                           float64
e_band/num_atoms             float64
e_coulomb/num_atoms          float64
e_entropy/num_atoms          float64
e_external/num_atoms           int64
e_kinetic/num_atoms          float64
e

# Preparing training data

In [8]:
for key, val in enumerate(df.columns):
    print(f"{key=}, {val=}")

key=0, val='formula_pretty'
key=1, val='formula_anonymous'
key=2, val='chemsys'
key=3, val='material_id'
key=4, val='structure'
key=5, val='formation_energy_per_atom'
key=6, val='is_metal'
key=7, val='e_total'
key=8, val='e_ionic'
key=9, val='e_electronic'
key=10, val='n'
key=11, val='fields_not_requested'
key=12, val='structure_cif'
key=13, val='e_band'
key=14, val='e_coulomb'
key=15, val='e_entropy'
key=16, val='e_external'
key=17, val='e_kinetic'
key=18, val='e_kinetic0'
key=19, val='e_xc'
key=20, val='e_total_free'
key=21, val='ef'
key=22, val='e_band/num_atoms'
key=23, val='e_coulomb/num_atoms'
key=24, val='e_entropy/num_atoms'
key=25, val='e_external/num_atoms'
key=26, val='e_kinetic/num_atoms'
key=27, val='e_kinetic0/num_atoms'
key=28, val='e_xc/num_atoms'
key=29, val='e_total_free/num_atoms'


In [19]:
feat = np.array(df.columns[13:22])
feat_norm = np.array(df.columns[21:])

In [20]:
dataset = df.copy()
dataset = dataset.dropna()
train = dataset.sample(frac=0.8, random_state=0)
test = dataset.drop(train.index)

print("Train shape:",train.shape)
print("Test shape:",test.shape)

Train shape: (797, 30)
Test shape: (199, 30)


In [21]:
# Label
Y_train = train.pop("formation_energy_per_atom")
Y_test = test.pop("formation_energy_per_atom")

# Input
X_train = train[feat]
X_test = test[feat]

X_train_norm = train[feat_norm]
X_test_norm = test[feat_norm]

# Traning model

## No hidden layer

In [24]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

# Define the input layer with the appropriate input shape
input_layer = tf.keras.layers.Input(shape=(len(feat),))

model = tf.keras.Sequential([
        input_layer,
        normalizer,
        tf.keras.layers.Dense(1)
    ])

model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

history = model.fit(
    np.array(X_train),
    np.array(Y_train),
    epochs=10,
    verbose=1,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)],
    batch_size=32
)

loss = history.history['val_loss'][-1]

Epoch 1/10
20/20 [==============================] - 1s 25ms/step - loss: 1.8822 - val_loss: 1.8699
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 1.8605 - val_loss: 1.8474
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 1.8392 - val_loss: 1.8250
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 1.8185 - val_loss: 1.8033
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 1.7983 - val_loss: 1.7812
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 1.7781 - val_loss: 1.7602
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 1.7576 - val_loss: 1.7378
Epoch 8/10
20/20 [==============================] - 0s 4ms/step - loss: 1.7376 - val_loss: 1.7170
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 1.7179 - val_loss: 1.6948
Epoch 10/10
20/20 [==============================] - 0s 4ms/step - loss: 1.6979 - val_loss: 1.6743


# One hidden layer

In [ ]:
neurons = [8, 16, 32, 64, 128, 256, 512]

results = {}
input_layer = tf.keras.layers.Input(shape=(len(feat),))

In [ ]:
# Define a list of different numbers of neurons and hidden layers
neurons = [8, 16, 32, 64, 128, 256]
hid_layers = [0, 1, 2, 3, 4, 5, 6]

# Create an empty dictionary to store the results
results = {}

# Create a function to build and train the model with given parameters
def build_and_train_model(num_neurons, num_hidden_layers):
    # Define the input layer with the appropriate input shape
    input_layer = tf.keras.layers.Input(shape=(len(feat_col),))

    # Initialize a list to store the hidden layers
    hidden_layers = []

    # Create the hidden layers
    for _ in range(num_hidden_layers):
        hidden_layers.append(tf.keras.layers.Dense(num_neurons, activation='relu'))

    # Define the model
    model = tf.keras.Sequential([input_layer] + hidden_layers + [tf.keras.layers.Dense(1)])

    # Compile the model
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

    # Train the model
    history = model.fit(
        np.array(train_dataset),
        np.array(train_labels),
        epochs=10,
        verbose=0,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)],
        batch_size=32
    )

    # Extract the final validation loss
    loss = history.history['val_loss'][-1]

    return loss

# Iterate through the number of hidden layers first
for num_hidden_layers in hid_layers:
    # Create a dictionary for the results of this number of hidden layers
    results[num_hidden_layers] = {}
    
    # Iterate through the different numbers of neurons
    for num_neurons in neurons:
        loss = build_and_train_model(num_neurons, num_hidden_layers)
        results[num_hidden_layers][num_neurons] = loss